In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout

# Set reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Generate date range from Jan 2019 to Jan 2025 (monthly)
date_range = pd.date_range(start="2019-01-01", end="2025-01-01", freq='MS')
num_months = len(date_range)

In [4]:

# Function to generate synthetic macroeconomic data
def generate_trend(base, growth_rate, noise_level=0.02):
    """Generates synthetic time-series data with a trend, seasonality, and noise."""
    trend = base * (1 + growth_rate) ** np.arange(num_months)
    seasonality = 1 + 0.05 * np.sin(np.linspace(0, 12 * np.pi, num_months))
    noise = np.random.normal(1, noise_level, num_months)
    return trend * seasonality * noise

In [5]:
# Define macroeconomic variables
variables = [
    "GDP Growth Rate (%)", "Inflation Rate (CPI, %)", "Federal Reserve Interest Rate (%)",
    "Unemployment Rate (%)", "Consumer Confidence Index", "Retail Sales Growth (%)",
    "Government Debt (% of GDP)", "Trade Balance (Billion $)", "Oil Price (WTI, $/barrel)",
    "Stock Market Index (S&P 500)", "Housing Market Index", "Business Investment Growth (%)",
    "Manufacturing PMI", "Supply Chain Index", "Labor Productivity Growth (%)",
    "Renewable Energy Share (%)", "Healthcare Expenditure (% of GDP)", "Real Estate Price Index",
    "Automotive Sales (Million Units)", "Defense Spending (Billion $)", "Fintech Transactions (Trillion $)",
    "E-commerce Share of Retail (%)", "Agricultural Output Index", "Freight Index",
    "Industrial Production Index", "Media Ad Revenue (Billion $)", "Tourism Revenue (Billion $)",
    "Telecom Broadband Subscriptions (Million)", "Mining Output Index"
]

In [6]:
# Create synthetic dataset
data = {"Date": date_range}
for var in variables:
    data[var] = generate_trend(
        np.random.uniform(1, 100), np.random.uniform(-0.002, 0.005), np.random.uniform(0.02, 0.1)
    )

In [7]:

# Convert to DataFrame
df = pd.DataFrame(data)

# Fill missing values using interpolation
df.interpolate(method='linear', inplace=True)


In [8]:

# Normalize data using MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[variables])


In [9]:
# Split into train, validation, and test sets
train_size = int(len(df_scaled) * 0.7)
val_size = int(len(df_scaled) * 0.15)
test_size = len(df_scaled) - train_size - val_size

train, val, test = df_scaled[:train_size], df_scaled[train_size:train_size+val_size], df_scaled[train_size+val_size:]


In [10]:

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])  # Shape: (seq_length, num_features)
        labels.append(data[i+seq_length])  # Shape: (num_features,)
    return np.array(sequences), np.array(labels)



In [11]:

seq_length = 9  # Use past 9 months to predict the next
X_train, y_train = create_sequences(train, seq_length)
X_val, y_val = create_sequences(val, seq_length)
X_test, y_test = create_sequences(test, seq_length)


In [12]:
# ✅ Fix shapes
X_train = X_train.reshape(X_train.shape[0], seq_length, len(variables))  # (samples, 12, 29)
X_val = X_val.reshape(X_val.shape[0], seq_length, len(variables))
X_test = X_test.reshape(X_test.shape[0], seq_length, len(variables))

y_train = y_train.reshape(y_train.shape[0], len(variables))  # (samples, 29)
y_val = y_val.reshape(y_val.shape[0], len(variables))
y_test = y_test.reshape(y_test.shape[0], len(variables))


In [13]:

# Debugging: Print shapes
print(f"X_train shape: {X_train.shape}")  # Expected: (num_samples, seq_length, num_features)
print(f"y_train shape: {y_train.shape}")  # Expected: (num_samples, num_features)


X_train shape: (42, 9, 29)
y_train shape: (42, 29)


In [14]:

# Debugging: Print shapes
print(f"X_train shape: {X_val.shape}")  # Expected: (num_samples, seq_length, num_features)
print(f"y_train shape: {y_val.shape}")  # Expected: (num_samples, num_features)


X_train shape: (1, 9, 29)
y_train shape: (1, 29)


In [15]:

# --- 🔵 Functional API Model ---
input_layer = Input(shape=(seq_length, len(variables)))

# First LSTM layer (returns sequences)
x = LSTM(100, activation='relu', return_sequences=True)(input_layer)
x = Dropout(0.2)(x)

# Second LSTM layer (returns final output)
x = LSTM(50, activation='relu')(x)
x = Dropout(0.2)(x)

# Output layer
output_layer = Dense(len(variables))(x)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

# --- 🔵 Compile Model with MAPE as Loss ---
model.compile(optimizer='adam', loss='mape')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 9, 29)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 9, 100)              │          52,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 9, 100)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          30,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 29)                  │           1,479 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 83,679 (326.87 KB)

 Trainable params: 83,679 (326.87 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:

# Train model on train data, validate on val data
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - loss: 699241.5625 - val_loss: 99.5097
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 413450.1562 - val_loss: 98.7807
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 419208.7500 - val_loss: 98.8708
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 167662.6094 - val_loss: 99.2033
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 184245.9062 - val_loss: 99.2355
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 110313.5703 - val_loss: 98.9888
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 99710.2734 - val_loss: 98.7411
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 81689.9297 - val_loss: 98.4237
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 61242.2500 - val_loss: 98.1898
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 78089.5000 - val_loss: 97.9139
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 81313.9688 - val_loss: 97.7041
Epoch 12/50
3/3 ━━━━━━━

In [17]:
# --- 🔵 Iterative Forecasting Function ---
def iterative_forecast(model, X_input, steps):
    """Performs iterative forecasting using rolling window approach."""
    forecasted_values = []
    current_input = X_input.copy()

    for _ in range(steps):
        prediction = model.predict(current_input.reshape(1, seq_length, len(variables)), verbose=0)
        forecasted_values.append(prediction.flatten())  # Store prediction
        current_input = np.roll(current_input, shift=-1, axis=0)  # Slide window
        current_input[-1] = prediction  # Replace last timestep with predicted values
    
    return np.array(forecasted_values)

In [18]:


# --- 🔵 Iterative Forecasting for Validation Data ---
val_forecasts = iterative_forecast(model, X_val[0], steps=len(y_val))

# Compute Validation MAPE
val_mape = np.mean(np.abs((y_val - val_forecasts) / y_val)) * 100
print(f"Validation MAPE:\n0 - val_loss: {val_mape:.4f}")


Validation MAPE:
0 - val_loss: 89.8115


In [19]:

# --- 🔵 Retrain on Full Train+Validation+Test Set ---
X_full_train = np.concatenate((X_train, X_val, X_test), axis=0)
y_full_train = np.concatenate((y_train, y_val, y_test), axis=0)
model.fit(X_full_train, y_full_train, epochs=50, batch_size=16)

# Perform iterative forecasting on test set
test_forecasts = iterative_forecast(model, X_test[0], steps=len(y_test))

# Compute Test MAPE
test_mape = np.mean(np.abs((y_test - test_forecasts) / y_test)) * 100
print(f"Test MAPE:\n0 - val_loss: {test_mape:.4f}")


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 111075.8203
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 108051.7109
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 101314.2188
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 97307.2812
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 98082.3906
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 99629.1641
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 95230.1172
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 94503.7422
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 88382.6094
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 81852.6562
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 75392.8047
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 75203.1562
Epoch 13/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 72956.3438
Epoch 14/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 69815.6016
Epoch 15/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 

<ipython-input-19-f95e20606400>:10: RuntimeWarning: divide by zero encountered in divide
  test_mape = np.mean(np.abs((y_test - test_forecasts) / y_test)) * 100


In [20]:


# --- 🔵 Forecast for Next 6 Months ---
future_forecasts = iterative_forecast(model, X_full_train[-1], steps=6)

# Convert forecasts into a DataFrame
forecast_dates = pd.date_range(start="2025-02-01", periods=6, freq="M")  # Adjust start date as needed
forecast_df = pd.DataFrame(future_forecasts, columns=variables, index=forecast_dates)
forecast_df.index.name = "Date"

# Print the forecast DataFrame
print("\n🔮 Forecast for Next 6 Months:")
##print(forecast_df)



# Save the final trained model
model.save("lstm_macro_forecast_final.h5")



🔮 Forecast for Next 6 Months:


<ipython-input-20-cc2504aae224>:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast_dates = pd.date_range(start="2025-02-01", periods=6, freq="M")  # Adjust start date as needed


In [21]:
# Save to CSV file
forecast_df.to_csv("6_month_forecast.csv", index=True)